In [11]:
#01 RGB
import cv2

def BGR2RGB(img):
    r = img[:, :, 2].copy()
    g = img[:, :, 1].copy()
    b = img[:, :, 0].copy()

    img[:,:,0] = r
    img[:,:,1] = g
    img[:,:,2] = b

    return img

img = cv2.imread("imori.jpg")
img = BGR2RGB(img)

# Save result
cv2.imwrite("out_rgb.jpg", img)
# cv2.imshow("result", img)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

True

In [12]:
#02 GrayScale
import cv2
import numpy as np

# astype 类型转换
def grayscale(img):
    b = img[:,:,0].copy()
    g = img[:,:,1].copy()
    r = img[:,:,2].copy()

    y = 0.2126*r + 0.7152*g + 0.0722*b #浮点运算，读取Img时转换成浮点类型

    y = y.astype(np.uint8)

    return y

img = cv2.imread("imori.jpg")
gray = grayscale(img)
cv2.imwrite("out_gray.jpg",gray)


True

In [13]:
#03 Binarization
import cv2
import numpy as np

# 二值化
def binarization(img,th=128):
    img[img < th] = 0
    img[img >= th ] = 255
    return img 

img = cv2.imread("out2.jpg")
t = binarization(img,th=128)
cv2.imwrite("out_binarization.jpg",t)


TypeError: '<' not supported between instances of 'NoneType' and 'int'

In [ ]:
#04 Otsu's Method
import cv2
import numpy as np

def otsu_binarization(gray_img):
    #最大类间方差算法
    max_t = 0
    max_sigma = 0
    H,W,_ = gray_img.shape

    for _t in range(1, 255):
        c0 = gray_img[np.where(gray_img < _t)]
        c1 = gray_img[np.where(gray_img >= _t)]

        w0 = len(c0)/(H*W)
        w1 = len(c1)/(H*W)

        m0 = np.mean(c0) if len(c0) > 0 else 0
        m1 = np.mean(c1) if len(c1) > 0 else 0

        sigma = w0*w1 *((m0-m1)**2)
        if sigma > max_sigma:
            max_sigma = sigma
            max_t = _t

    print("threshold >>", max_t)
    th = max_t
    gray_img[gray_img < th] = 0
    gray_img[gray_img >= th] = 255

    return gray_img

img = cv2.imread("out_gray.jpg")
out = otsu_binarization(img)
cv2.imwrite("out_otsu.jpg",out)



In [1]:
#05 HSV变换
import cv2 
import numpy as np

def BGR2HSV(_img):
    #RGB的取值范围为0到1
    img = _img.copy() / 255.

    hsv = np.zeros_like(img,dtype = np.float32)

    #取RGB中最大的,min_arg找到最小的那一维
    max_v = np.max(img, axis=2).copy()
    min_v = np.min(img, axis=2).copy()
    min_arg = np.argmin(img, axis=2)

    #H
    hsv[...,0][np.where(max_v == min_v)] = 0
    ## Min=B
    #np.where返回满足条件的坐标
    ind = np.where(min_v == 0)
    hsv[...,0][ind] = 60 * (img[...,1][ind]-img[...,2][ind]) / (max_v[ind]-min_v[ind]) + 60
    ## Min=R
    ind = np.where(min_v == 2)
    hsv[...,0][ind] = 60 * (img[...,0][ind]-img[...,1][ind]) / (max_v[ind]-min_v[ind]) + 180
    ## Min=G
    ind = np.where(min_v == 1)
    hsv[...,0][ind] = 60 * (img[...,2][ind]-img[...,0][ind]) / (max_v[ind]-min_v[ind]) + 300

    #S
    hsv[...,1] = max_v.copy() - min_v.copy()

    #V
    hsv[...,2] = max_v.copy()

    return hsv

def HSV2BGR(_img,hsv):
    img =_img.copy() / 255.

    max_v = np.max(img, axis=2).copy()
    min_v = np.min(img, axis =2).copy()

    out = np.zeros_like(img)

    H = hsv[...,0]
    S = hsv[...,1]
    V = hsv[...,2]
   
    C = S
    H_ = H/60.
    X = C * (1 - np.abs( H_%2 - 1))
    Z = np.zeros_like(H)

    vals = [[Z,X,C], [Z,C,X], [X,C,Z], [C,X,Z], [C,Z,X], [X,Z,C]]

    for i in range(6):
        ind = np.where((i <= H_) & (H_ < (i+1)))
        out[...,0][ind] = (V - C)[ind] + vals[i][0][ind]
        out[...,1][ind] = (V - C)[ind] + vals[i][1][ind]
        out[...,2][ind] = (V - C)[ind] + vals[i][2][ind]
    out[np.where(max_v == min_v)] = 0
    out = np.clip(out, 0 , 1)
    out = (out * 255).astype(np.uint8)

    return out

img = cv2.imread("imori.jpg").astype(np.float32)

# RGB > HSV
hsv = BGR2HSV(img)

# 将色相反转，再用RGB
# Transpose Hue
hsv[..., 0] = (hsv[..., 0] + 180) % 360

# HSV > RGB
out = HSV2BGR(img, hsv)

# Save result
cv2.imwrite("out_hsvrgb.jpg", hsv)
cv2.imshow("result", hsv)
cv2.waitKey(0)
cv2.destroyAllWindows()